# Building RAG from Scratch



This notebook walks through the steps to build a RAG model from scratch, using the Llama 2 model as an example. We will walk through the following steps:

1. **Setup the environment**: Sentence Transformers as the embedding model, Llama CPP as the LLM, and Postgres as the vector store.
2. **Build an ingestion pipeline**: load documents, split into text chunks, generate embeddings, and load into the vector store.
3. **Build a retrieval pipeline**: generate a query embedding, query the vector store, parse the results, and put into a retriever.

![Rag Pipeline](../static/img/0_rag_pipeline.jpg)

# 1. Setup

## 1.1 Sentence Transformers

In [1]:
# sentence transformers
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

## 1.2 Llama CPP

In [2]:
!pip install llama-cpp-python==0.2.32

In [3]:
from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/alex/cache_files/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

## 1.3 Define Service Context

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

## 1.4 Initialize Postgres

Setup a postgres database on MacOS: [Setting up a PostgreSQL Database on Mac](https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/). Then Add a role like the following:

```sql
   CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
   ALTER ROLE <user> SUPERUSER;
```

In [5]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.0/665.0 kB 1.9 MB/s eta 0:00:0000:0100:01


In [5]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "password"  # change to your password
port = "5432"
user = "alex"  # change to your user
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [26]:
from sqlalchemy import make_url
from llama_index.vector_stores import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

# 2. Build an Ingestion Pipeline

## 2.1 Load Data

In [8]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

--2024-01-24 12:16:08--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  3.05MB/s    in 4.8s    

2024-01-24 12:16:13 (2.69 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [7]:
from pathlib import Path
from llama_hub.file.pymu_pdf.base import PyMuPDFReader

In [8]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

## 2.2 Split Documents

In this notebook, we will split the documents into chunks of 1024.

In practice, you may want to split into different chunks with different methods, because:

- relevant and semantically rich, improving the efficiency and accuracy
- sentences focus on specific meanings, while larger texts capture broader contexts, with trade-offs in terms of noise and precision
- shorter queries aligning better with sentence-level embeddings and longer ones requiring paragraph or document-level embeddings

There are a lot **Chunking Strategies**, including:
- **Fixed-size** is best for uniformity and simplicity, but may lost context and not adaptive.
- **Sentence splitting** is better for semantic relevance and may not capture broader context well.
- **Recursive chunking** is an iterative method that adapts to the content, but may result in uneven chunk sizes.
- **Specialized chunking** is used for structured content formats to maintain formatting and structure, but requires specific methods for different formats.

More details can be found in [Chunking Strategies for LLM Applications](https://www.pinecone.io/learn/chunking-strategies/)

In [9]:
from llama_index.node_parser.text import SentenceSplitter

In [10]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [11]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

## 2.3 Construct Nodes from Text Chunks

A **Node** is a representation of a segment from a source Document. This segment could be a piece of text, an image, or any other type of content. Like Documents, Nodes also **carry metadata** and information about their relationships with other nodes.

In [12]:
from llama_index.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)


## 2.4 Generate Embeddings for each Node

In [13]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


## 2.5 Load Nodes into a Vector Store

insert these nodes into our PostgresVectorStore

In [14]:
vector_store.add(nodes)

['72d428b0-34ec-4302-8fb6-0fa7432221a7',
 'b9b2ba82-5ed2-4673-aca3-a673df40c8f6',
 '23911d78-faa5-4901-901e-701f36f0633a',
 '58dd3296-edaa-4026-9cab-d830724876e1',
 '52cda33f-2994-47ed-bc45-9841089f47b2',
 '781d6315-538f-4de7-970c-04fcbf858b2c',
 'd58cf39e-edb3-495c-8826-7d81d31dd8bd',
 '7205cfad-9eff-4936-bb21-afff28c823a3',
 'c60412cc-22dc-4bf0-b963-51082d909002',
 '67b02abe-4827-4750-b149-3374cce9821d',
 'c271681c-b937-4d82-8b1b-697a9bdc3444',
 'd67a2a15-5a5e-4328-8500-d701755409e2',
 '26727e9e-656d-4a3a-94c1-027db686eb88',
 '60792374-e6c1-4c86-8ac5-ef2873cea6d2',
 '31ac73dd-5a68-4f23-aeec-a8893433e5f2',
 '9015aca5-c779-4716-a367-00e297720611',
 '55bc9afb-9f26-4c0f-bf6b-e7cf60a61556',
 '351fcf0e-8667-42ae-ab3c-06a473f99a8e',
 'e73224db-95a9-435f-9445-db537dd9a837',
 '02ccc30a-4143-47fb-ad0b-d4243fc37119',
 '5676ea27-61d0-43df-82f5-592a852e1799',
 'e71cb5d0-f0d9-4d66-9bb4-c3cc7520b8ee',
 'adc0e273-2e7b-4c2e-a3e2-364a82159e6b',
 '70aac457-9e60-4921-8763-62ea304bf8bc',
 '39ceb2c0-e1cf-

# 3. Build Retrieval Pipeline

In [15]:
query_str = "Can you tell me about the key concepts for safety finetuning"

## 3.1 Generate Embedding for Query

In [16]:
query_embedding = embed_model.get_query_embedding(query_str)

## 3.2 Query the Vector Database

In [17]:
# construct vector store query
from llama_index.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [18]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

## 3.3 Parse Result into a Set of Nodes

In [19]:
from llama_index.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

## 3.4 Put into a Retriever

In [20]:
from llama_index import QueryBundle
from llama_index.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [21]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)


# 4. Use RetrieverQueryEngine to synthesize a response

In [22]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever, service_context=service_context
)


In [23]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   30001.45 ms
llama_print_timings:      sample time =      11.71 ms /   117 runs   (    0.10 ms per token,  9987.20 tokens per second)
llama_print_timings: prompt eval time =   73055.23 ms /  1850 tokens (   39.49 ms per token,    25.32 tokens per second)
llama_print_timings:        eval time =   18684.30 ms /   116 runs   (  161.07 ms per token,     6.21 tokens per second)
llama_print_timings:       total time =   91919.94 ms /  1966 tokens


In [24]:
print(str(response))

 Based on the results presented in Table 3, Llama 2 outperforms all open-source models on
most of the benchmarks. Specifically, Llama 2 70B model improves the results on MMLU and BBH by ≈5 and
≈8 points, respectively, compared to the best open-source model (GPT-3.5). Additionally, Llama 2 7B and 30B
models outperform MPT models of the corresponding size on all categories besides code benchmarks.


In [25]:
print(response.source_nodes[0].get_content())

Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI

# Reference
* https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html
* https://www.pinecone.io/learn/chunking-strategies/